In [ ]:
""" File to generate an LPIPS attack in the infinity norm 
(specifically to beat the Madry challenge)

Steps to get this done:
-1) Import a buncha things 
 0) Load up my dataset, normalizer, adversarially trained net 
 1) Build attack parameters
 2) Check efficacy on small dataset 
 3) Build madry dataset 
"""
print


In [ ]:
# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch.nn as nn

import config
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import checkpoints

In [ ]:
# Block 0: load dataset, normalizer, adversarially trained net 
val_loader = cifar_loader.load_cifar_data('val', normalize=False, batch_size=16, use_gpu=True)

cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS,
                                           std=config.CIFAR10_STDS)

base_model = cifar_resnets.resnet32()
adv_trained_net = checkpoints.load_state_dict_from_filename('half_trained_madry.th', base_model)
eval_obj = adveval.AdversarialEvaluation(adv_trained_net, cifar_normer)



In [ ]:
# Block 1: build attack parameters 
ATTACK_KWARGS = {'l_inf_bound': 8.0/255.0, 
                 'step_size': 0.5/255.0,
                 'num_iterations': 20, 
                 'random_init': True, 
                 'signed': True, 
                 'verbose': False}
ATTACK_SPECIFIC_PARAMS = {'attack_kwargs': ATTACK_KWARGS}

def build_attack_loss(classifier, normalizer, lpips_penalty):
    """ Builds a regularized loss function for use in PGD 
    Takes in (perturbed_examples, labels) and returns 
    XEntropy(perturbed_examples, labels) + hyperparam * LPIPS(examples, perturbed_examples)
    """    
    return plf.PerceptualXentropy(classifier, normalizer=normalizer, 
                                  regularization_constant=lpips_penalty, 
                                  use_gpu=True)

attack_params = {}
penalties = [0.01, 0.1, 1.0, 10.0, 100.0]
for penalty in penalties:
    loss_obj = build_attack_loss(adv_trained_net, cifar_normer, penalty)
    attack_obj = aa.LInfPGD(adv_trained_net, cifar_normer, loss_obj, use_gpu=True)
    attack_param = advtrain.AdversarialAttackParameters(attack_obj, 1.0, 
                                                        attack_specific_params=ATTACK_SPECIFIC_PARAMS)
    attack_params[str(penalty)] = attack_param 

 
#eval_obj.evaluate_ensemble(val_loader, {'perceptual': attack_params['1.0']}, use_gpu=True)

single_loss = build_attack_loss(adv_trained_net, cifar_normer, 1.0)
                                

# single_loss.losses['lpips_reg'].dist_model.zero_grad()

In [ ]:
# Try and forward back on single loss calls to break thing 
from torch.autograd import Variable
def forward_pass(epochs):
    grad_sum = 0
    single_loss = build_attack_loss(adv_trained_net, cifar_normer, 1.0).losses['lpips_reg']
    
    val_loader = cifar_loader.load_cifar_data('val', normalize=False, batch_size=16, use_gpu=True)
    first_batch, _ = next(iter(val_loader))
    first_batch_var = Variable(first_batch.cuda(), requires_grad=False)
    single_loss.setup_attack_batch(first_batch_var)
    
    for epoch in xrange(epochs):
        print epoch
        for batch, labels in val_loader:
            var_batch = Variable(batch.cuda(), requires_grad=True)
            out = torch.sum(single_loss.forward(var_batch))
            out.backward()
            grad_sum = grad_sum + var_batch.grad.data
        return grad_sum 
            
            


var_batch = forward_pass(10)


In [ ]:
var_batch

In [ ]:
# Self contained memout assertion 

##########################################################################
#   IMPORT BLOCK                                                         #
##########################################################################
import gc 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch.nn as nn

import config
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import checkpoints

##########################################################################
#   FUNCTION BLOCK                                                       #
##########################################################################
def memout_example():    
    assert vars() == {}
    # empty slate 

    # build persistent data 
    val_loader = cifar_loader.load_cifar_data('val', normalize=False, batch_size=16, use_gpu=True)
    
    # now loop through batches and show that there's something hanging somewhere...
    for batch_no, (batch, labels) in enumerate(val_loader):
        
        # clean up garbage and clear cuda cache as much as possible 
        gc.collect()
        print "BATCH NUMBER: %s" % batch_no
        assert sorted(vars().keys()) == sorted(['labels', 'val_loader', 'batch', 'batch_no'])
        torch.cuda.empty_cache()
        
        
        # load things needed for attack 
        base_model = cifar_resnets.resnet32()
        adv_trained_net = checkpoints.load_state_dict_from_filename('half_trained_madry.th', base_model)
        adv_trained_net.cuda()
        cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS,
                                           std=config.CIFAR10_STDS)        
        pgd_perceptual_loss = plf.PerceptualXentropy(adv_trained_net, normalizer=cifar_normer, use_gpu=True)            
        pgd_attack_obj = aa.LInfPGD(adv_trained_net, cifar_normer, pgd_perceptual_loss)
        
        adv_images = pgd_attack_obj.attack(batch.cuda(), labels.cuda(), l_inf_bound =8.0/255.0, 
                                           step_size=1.0/255.0, num_iterations=16, verbose=False)
        
        # push things to cpu (in hopes it gets them out of the cache)
        # also delete everything and be sure to collect garbage before next batch 
        batch.cpu()
        labels.cpu()
        del adv_images
        del batch 
        del labels 
        del pgd_attack_obj 
        del pgd_perceptual_loss
        del cifar_normer
        adv_trained_net.cpu()
        del adv_trained_net 
        del base_model 
    return
    
    
##########################################################################
#   BREAK THE PLANET BLOCK                                               #
##########################################################################
print memout_example()
print "SOMEHOW THIS WORKED??"



In [ ]:
adv_images.c

In [ ]:
# UGHHHHH MEMORY LEAK HELL
# So first show that vanilla Xentropy is fine 
# Then show that L2 norms are fine 
# Then isolate that LPIPS is breaking things 

# Step 1: vanilla xentropy 
vanilla_loss = plf.VanillaXentropy(adv_trained_net, normalizer=cifar_normer)
attack_obj = aa.LInfPGD(adv_trained_net, cifar_normer, vanilla_loss, use_gpu=True)
vanilla_param = advtrain.AdversarialAttackParameters(attack_obj, 1.0, attack_specific_params=ATTACK_SPECIFIC_PARAMS)

output = eval_obj.evaluate_ensemble(val_loader, {'vanilla': vanilla_param}, use_gpu=True)




In [ ]:

# Step 2: l2 norms 
# --- make l2 + xentropy loss 
class L2Xentropy(lf.RegularizedLoss):
    """ Xentropy loss with a regularization based on l2 distance """

    def __init__(self, classifier, normalizer=None,
                 regularization_constant=-100.0):
        partial_xentropy = lf.PartialXentropy(classifier, normalizer=normalizer)
        l2_reg = lf.L2Regularization(None)

        super(L2Xentropy, self).__init__({'xentropy': partial_xentropy,
                                                  'l2_reg': l2_reg},
                                                  {'xentropy': 1.0,
                                                   'l2_reg':
                                                       regularization_constant})
# --- /make l2 + xentropy loss
l2_loss = L2Xentropy(adv_trained_net, normalizer=cifar_normer)
l2_attack_obj = aa.LInfPGD(adv_trained_net, cifar_normer, l2_loss, use_gpu=True)
l2_param = advtrain.AdversarialAttackParameters(l2_attack_obj, 1.0, attack_specific_params=ATTACK_SPECIFIC_PARAMS)
output = eval_obj.evaluate_ensemble(val_loader, {'l2': l2_param}, use_gpu=True)



In [ ]:
mini_ex, mini_label = next(iter(val_loader))

In [ ]:
from torch.autograd import Variable
# print l2_loss.losses['l2_reg'].fix_im.is_cuda
vini_ex = Variable(mini_ex.cuda(), requires_grad=True)
vini_label = Variable(mini_label.cuda(), requires_grad=False)

loss_out = l2_loss.forward(vini_ex, vini_label)

print loss_out
